In [ ]:
USER_FLAG = "--user"
!pip3 install --user google-cloud-aiplatform==1.18.1
#!pip3 install  --upgrade google-cloud-aiplatform 
#!pip3 install -U google-cloud-storage  
!pip3 install --user google-cloud-pipeline-components
! pip3 install --user -U google-cloud-storage 
!pip3 install --user  kfp==1.8.9

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [ ]:
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID


In [ ]:
REGION = "us-central1"

In [ ]:
BUCKET_NAME = "gs://{}-bucket-ai-pip".format(PROJECT_ID)
!gsutil mb -l us-central1 $BUCKET_NAME # You only need to run this once
BUCKET_URI = f"gs://{BUCKET_NAME}/lightwght-pip-root/"
PIPELINE_ROOT = f"{BUCKET_NAME}/lightwght-pip-root/"


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# We'll use this beta library for metadata querying
from google.cloud import aiplatform as aip, aiplatform_v1beta1 as aipbeta
from google.cloud.aiplatform import pipeline_jobs 
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import (Artifact, Dataset, Input,InputPath, Metrics, Model, Output,
                        OutputPath, component)
from google.oauth2 import service_account as sa
from typing import NamedTuple

In [ ]:
PATH = get_ipython().run_line_magic("env", "PATH")
%env PATH={PATH}:/home/jupyter/.local/bin
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)
shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID: ", PROJECT_ID)

In [ ]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
!export GOOGLE_APPLICATION_CREDENTIALS="/home/jupyter/src/myplayground/raniamoh-playground-ai.json"
!gcloud auth activate-service-account ai-piplinesa@raniamoh-playground.iam.gserviceaccount.com --key-file="/home/jupyter/src/myplayground/raniamoh-playground-ai.json"
!gcloud iam service-accounts add-iam-policy-binding 189908348872-compute@developer.gserviceaccount.com  --member=serviceAccount:ai-piplinesa@raniamoh-playground.iam.gserviceaccount.com  --role=roles/iam.serviceAccountUser --project=raniamoh-playground
!gcloud iam service-accounts add-iam-policy-binding ai-piplinesa@raniamoh-playground.iam.gserviceaccount.com  --member=serviceAccount:ai-piplinesa@raniamoh-playground.iam.gserviceaccount.com  --role=roles/iam.serviceAccountUser --project=raniamoh-playground

In [ ]:
aip.init(project=PROJECT_ID, location=REGION,  staging_bucket=BUCKET_URI, credentials=sa.Credentials.from_service_account_file(
    '/home/jupyter/src/myplayground/raniamoh-playground-ai.json'))

In [ ]:
@component
def preprocess(
    # An input parameter of type string.
    message: str,
    # Use Output to get a metadata-rich handle to the output artifact
    # of type `Dataset`.
    output_dataset_one: Output[Dataset],
    # A locally accessible filepath for another output artifact of type
    # `Dataset`.
    output_dataset_two_path: OutputPath("Dataset"),
    # A locally accessible filepath for an output parameter of type string.
    output_parameter_path: OutputPath(str),
):
    """'Mock' preprocessing step.
    Writes out the passed in message to the output "Dataset"s and the output message.
    """
    output_dataset_one.metadata["hello"] = "there"
    # Use OutputArtifact.path to access a local file path for writing.
    # One can also use OutputArtifact.uri to access the actual URI file path.
    with open(output_dataset_one.path, "w") as f:
        f.write(message)

    # OutputPath is used to just pass the local file path of the output artifact
    # to the function.
    with open(output_dataset_two_path, "w") as f:
        f.write(message)

    with open(output_parameter_path, "w") as f:
        f.write(message)

In [ ]:
@component(
    base_image="python:3.9",  # Use a different base image.
)
def train(
    # An input parameter of type string.
    message: str,
    # Use InputPath to get a locally accessible path for the input artifact
    # of type `Dataset`.
    dataset_one_path: InputPath("Dataset"),
    # Use InputArtifact to get a metadata-rich handle to the input artifact
    # of type `Dataset`.
    dataset_two: Input[Dataset],
    # Output artifact of type Model.
    imported_dataset: Input[Dataset],
    model: Output[Model],
    # An input parameter of type int with a default value.
    num_steps: int = 3,
    # Use NamedTuple to return either artifacts or parameters.
    # When returning artifacts like this, return the contents of
    # the artifact. The assumption here is that this return value
    # fits in memory.
) -> NamedTuple(
    "Outputs",
    [
        ("output_message", str),  # Return parameter.
        ("generic_artifact", Artifact),  # Return generic Artifact.
    ],
):
    """'Mock' Training step.
    Combines the contents of dataset_one and dataset_two into the
    output Model.
    Constructs a new output_message consisting of message repeated num_steps times.
    """

    # Directly access the passed in GCS URI as a local file (uses GCSFuse).
    with open(dataset_one_path, "r") as input_file:
        dataset_one_contents = input_file.read()

    # dataset_two is an Artifact handle. Use dataset_two.path to get a
    # local file path (uses GCSFuse).
    # Alternately, use dataset_two.uri to access the GCS URI directly.
    with open(dataset_two.path, "r") as input_file:
        dataset_two_contents = input_file.read()

    with open(model.path, "w") as f:
        f.write("My Model")

    with open(imported_dataset.path, "r") as f:
        data = f.read()
    print("Imported Dataset:", data)

    # Use model.get() to get a Model artifact, which has a .metadata dictionary
    # to store arbitrary metadata for the output artifact. This metadata will be
    # recorded in Managed Metadata and can be queried later. It will also show up
    # in the UI.
    model.metadata["accuracy"] = 0.9
    model.metadata["framework"] = "Tensorflow"
    model.metadata["time_to_train_in_seconds"] = 257

    artifact_contents = "{}\n{}".format(dataset_one_contents, dataset_two_contents)
    output_message = " ".join([message for _ in range(num_steps)])
    return (output_message, artifact_contents)

In [ ]:
@component
def read_artifact_input(
    generic: Input[Artifact],
):
    with open(generic.path, "r") as input_file:
        generic_contents = input_file.read()
        print(f"generic contents: {generic_contents}")

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="metadata-pipeline-v2",
)
def pipeline(message: str):
    importer = dsl.importer(
        artifact_uri="gs://ml-pipeline-playground/shakespeare1.txt",
        artifact_class=Dataset,
        reimport=False,
    )
    preprocess_task = preprocess(message=message)
    train_task = train(
        dataset_one_path=preprocess_task.outputs["output_dataset_one"],
        dataset_two=preprocess_task.outputs["output_dataset_two_path"],
        imported_dataset=importer.output,
        message=preprocess_task.outputs["output_parameter_path"],
        num_steps=5,
    )
    read_task = read_artifact_input(  # noqa: F841
        train_task.outputs["generic_artifact"]
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="lightweight_pipeline.json"
)

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

DISPLAY_NAME = "shakespeare_" + TIMESTAMP

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="lightweight_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"message": "Hello, World"},
)

job.run()

#! rm lightweight_pipeline.json